## <span style='font-family:Georgia'> Objectives
The purpose of this notebook is creating a baseline rule-model, which is to be a benchmark of the neural model developed in the later phase of the project.

In [1]:
# loading packages
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import copy
import matplotlib.pyplot as plt

sns.set(style="darkgrid")
from tqdm import tqdm

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from supportive_functions import count_punctuation, rm_consecutive_spaces

import nltk
import nltk.data
from polish_sentence_nltk_tokenizer import sentence_tokenizer

import warnings

warnings.filterwarnings(action="ignore")
pd.options.display.max_columns = None

%matplotlib inline
plt.rcParams["figure.figsize"] = (9, 6)

sns.set(
    rc={
        "figure.figsize": (14, 8.27),
        "axes.facecolor": "white",
        "axes.grid": True,
        "grid.color": ".9",
    }
)

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [2]:
def read_files(dir_path: str, sep: str = ",") -> pd.DataFrame:
    """
    Read the files with timestamps from a given catalog.

    Args:
        dir_path (str): Path to the catalog with the files
        sep (str, optional): Separator used in pd.read_csv() function. Defaults to ",".

    Returns:
        pd.DataFrame: Data from all the files concatenated into one dataframe.
    """
    files = os.listdir(dir_path)
    data = pd.DataFrame(
        [], columns=["index", "timestamp_start", "timestamp_stop", "word"]
    )
    for file in tqdm(files):
        f = open(os.path.join(dir_path, file), encoding="utf-8", mode="r")
        name = file.split(".")[0]
        df = pd.read_csv(f, header=None, sep=sep, encoding="utf-8")
        df["index"] = name
        if sep != ",":
            df["timestamp_start"] = df.iloc[:, 0].str.split(",").str[0].str[1:]
            df["timestamp_stop"] = (
                df.iloc[:, 0].str.split(",").str[1].str.split("\)").str[0]
            )
            df["word"] = df.iloc[:, 0].str.split("\)\s").str[1]
            df.drop([0], axis=1, inplace=True)
            df.drop(df.tail(1).index, inplace=True)
        else:
            df["timestamp_start"] = df.iloc[:, 0].str[1:]
            df["timestamp_stop"] = df.iloc[:, 1].str.split("\) \s").str[0]
            df["word"] = df.iloc[:, 1].str.split("\)\s").str[1]

            df.drop([0, 1], axis=1, inplace=True)
            df.drop(df.tail(1).index, inplace=True)
        data = pd.concat([data, df])
        f.close()
    return data

## <span style='font-family:Georgia'> Implementation of rules

In [3]:
# tokenizer for separation into sentences
tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

In [4]:
# extracting list of abbreviations
abbreviations = sentence_tokenizer._params.abbrev_types
abbreviations_with_period = [x for x in abbreviations if "." in x]

In [5]:
# separating abbreviations with periods inside them
spaced_abbreviations_with_period = []
for abbr in abbreviations_with_period:
    if "." in abbr:
        spaced_abbreviations_with_period.append(abbr.replace(".", " "))

In [6]:
# list of abbreviations after 'cleaning'
abbreviations = [
    abbr for abbr in abbreviations if abbr not in abbreviations_with_period
]
for abbr in spaced_abbreviations_with_period:
    abbreviations.append(abbr)

In [7]:
punctuation = list(sentence_tokenizer.PUNCTUATION)
punctuation.append('-')
punctuation.remove(';')

In [8]:
# reading dict generated from morfeusz (all items from train dataset)
morfeusz_train = pd.read_csv(
    "../../data/outputs/feature_engineering/morfeusz/items_dict.csv"
)
morfeusz_train.columns

Index(['item', 'grammatical_class'], dtype='object')

In [9]:
# list of conjuctions from morfeusz
conj_comp_list = morfeusz_train[
    morfeusz_train.grammatical_class.str.contains("comp|conj")
].item.tolist()

In [10]:
# all conjuctions below are from conj_comp_list
# not_conj - we do not put coma before them
# yes_conj - we do put coma before them

not_conj = [
    "a",
    "albo",
    "ali",
    "ani",
    "aniżeli",
    "aż",
    "bowiem",
    "bylem",
    "bym",
    "byśmy",
    "bądź",
    "co",
    "com",
    "coś",
    "czyli",
    "czym",
    "ewentualnie",
    "gdybyś",
    "gdybyśmy",
    "i",
    "inaczej",
    "jednak",
    "kiedyś",
    "lub",
    "miast",
    "miasto",
    "natomiast",
    "ni",
    "niby",
    "niczym",
    "nie",
    "nim",
    "niż",
    "niżem",
    "oraz",
    "tedy",
    "to",
    "tom",
    "toteż",
    "tudzież",
    "tylko",
    "tym",
    "tymczasem",
    "zamiast",
    "zarówno",
    "zatem",
    "zaś",
]
yes_conj = [
    "aby",
    "abyśmy",
    "acz",
    "aczkolwiek",
    "bo",
    "by",
    "chociaż",
    "chociażby",
    "choć",
    "choćby",
    "czy",
    "dopóki",
    "gdy",
    "gdyby",
    "gdyż",
    "im",
    "iż",
    "jak",
    "jakby",
    "jako",
    "jakoby",
    "jeśli",
    "jeżeli",
    "kiedy",
    "ledwo",
    "ponieważ",
    "póki",
    "skoro",
    "więc",
    "zaledwie",
    "zanim",
    "że",
    "żeby",
    "żebyś",
    "żeś",
    "ale",
    "lecz",
]

In [11]:
# list of który która które
ktory_list = morfeusz_train[morfeusz_train.item.str.startswith("któr")][
    ~morfeusz_train.item.str.contains("ś")
].item.tolist()
ktory_list.remove("którykolwiek")
ktory_list.remove("którędy")

In [12]:
#additional list of conjunctions with coma which were not in morfeusz
additional_list=['dlatego']

In [13]:
#merge lists of conjuctions
con_list_yes=yes_conj+ktory_list+additional_list

In [14]:
### train symbols and noisy words
# load list of symbols to replace
symbols_to_replace_infile = open(
    "../../data/outputs/eda/symbols_to_replace.txt", "r", encoding="utf-8"
)
symbols_to_replace = symbols_to_replace_infile.read().splitlines()
# load list of noisy words, i.e. words with letters from outside the Polish alphabet
noisy_words_infile = open(
    "../../data/outputs/eda/train/noisy_words.txt", "r", encoding="utf-8"
)
noisy_words = noisy_words_infile.read().splitlines()
# load list of letters from outside the Polish alphabet
non_polish_letters_infile = open(
    "../../data/outputs/eda/train/non_polish_letters.txt", "r", encoding="utf-8"
)
non_polish_letters = non_polish_letters_infile.read().splitlines()

# merge noisy data into one list
symbols_to_replace.extend(noisy_words)
symbols_to_replace.extend(non_polish_letters)

In [15]:
### test symbols and noisy words
# load list of symbols to replace
symbols_to_replace_infile_test = open(
    "../../data/outputs/eda/symbols_to_replace.txt", "r", encoding="utf-8"
)
symbols_to_replace_test = symbols_to_replace_infile_test.read().splitlines()
# load list of noisy words, i.e. words with letters from outside the Polish alphabet
noisy_words_infile_test = open(
    "../../data/outputs/eda/test/noisy_words.txt", "r", encoding="utf-8"
)
noisy_words_test = noisy_words_infile_test.read().splitlines()
# load list of letters from outside the Polish alphabet
non_polish_letters_infile_test = open(
    "../../data/outputs/eda/test/non_polish_letters.txt", "r", encoding="utf-8"
)
non_polish_letters_test = non_polish_letters_infile_test.read().splitlines()
# merge noisy data into one list
noisy_words_test.extend(non_polish_letters_test)

### <span style='font-family:Georgia'> Train data
#### <span style='font-family:Georgia'> Adding fullstops

In [16]:
data_train = read_files("../../data/source/poleval_fa.train/train", sep="\t")
data_train_calc = data_train.drop(["timestamp_start", "timestamp_stop"], axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 793/793 [00:15<00:00, 51.17it/s]


In [17]:
data_train

,index,timestamp_start,timestamp_stop,word
0,wikinews178430,690,750,we
1,wikinews178430,840,1350,wrocławiu
2,wikinews178430,1650,1920,walkę
3,wikinews178430,2130,2250,ze
4,wikinews178430,2310,2790,szkodnikiem
...,...,...,...,...
149,wikitalks109264,68760,69270,operacji
150,wikitalks109264,69390,69690,łódzkiej
151,wikitalks109264,70320,70320,w
152,wikitalks109264,70380,70530,roku


In [18]:
data_train_calc_joined = (
    data_train_calc[["index", "word"]]
    .groupby(["index"])["word"]
    .agg(" ".join)
    .reset_index()
)

In [19]:
for symb in symbols_to_replace:
    data_train_calc_joined["word"] = data_train_calc_joined["word"].apply(
        lambda x: x.replace(symb, "")
    )

data_train_calc_joined["word"] = data_train_calc_joined["word"].apply(
    rm_consecutive_spaces
)
data_train_calc_joined.columns = ["FileId", "FixedOutput"]
data_train = copy.deepcopy(data_train_calc_joined)

In [20]:
data_train.shape

(793, 2)

In [21]:
data_train.loc[0, 'FixedOutput']

'we wrocławiu walkę ze szkodnikiem rozpoczyna zespół szkół budowlanych przy ul grabiszyńskiej do akcji na ratunek kasztanowcom zsb zachęcił pobliskie szkoły gimnazjum nr 6 przy al pracy sp nr 82 przy ul blacharskiej sp nr 109 przy ul inżynierskiej sp nr 15 przy ul solskiego gimnazjum nr 40 przy ul morelowskiego pomoc ma polegać na szukaniu sponsorów którzy sfinansują specjalistyczne szczepienia koszt szczepienia jednego drzewa to ok 70 zł vat sponsor który uratuje kasztanowca ma zapewnioną na nim tabliczkę reklamową podobna akcja odbyła się w świdnicy gdzie szczepionkę dla 99 drzew zasponsorowała firma imp comfort która kupiła 10 ha gruntów w mieście i zamierza wybudować tam swoją fabrykę szrotówek kasztanowcowiaczek już od jakiegoś czasu niszczy kasztanowce do niedawna uważano że tego szkodnika można wytępić tylko poprzez palenie i kompostowanie liści kasztanowca teraz wiemy że możemy je uratować poprzez specjalne szczepionki jeśli możesz pomóc wrocławskim kasztanowcom to skontaktuj s

#### <span style='font-family:Georgia'> Inserting fullstops in abbreviations

In [22]:
# inserting fullstops after abbreviations + removing doubled spaces
for abbr in abbreviations:
    data_train['FixedOutput'] = data_train['FixedOutput'].apply(
        lambda x: x.replace(
            ' ' + abbr + ' ',
            ' ' + abbr.replace(' ', ' . ') + ' . ')
    )
data_train['FixedOutput'] = data_train['FixedOutput'].apply(
    lambda x: x.replace('. .', '.')
)

In [23]:
data_train.head()

,FileId,FixedOutput
0,wikinews178430,we wrocławiu walkę ze szkodnikiem rozpoczyna z...
1,wikinews178747,do serii rozbojów doszło w środę wieczorem w l...
2,wikinews178788,trzech 19 latków zatrzymała wczoraj lubelska p...
3,wikinews178804,drugie zwycięstwo w tegorocznym giro ditalia o...
4,wikinews178814,kradzieże telefonów komórkowych to coraz częst...


In [24]:
data_train.loc[0, 'FixedOutput']

'we wrocławiu walkę ze szkodnikiem rozpoczyna zespół szkół budowlanych przy ul . grabiszyńskiej do akcji na ratunek kasztanowcom zsb zachęcił pobliskie szkoły gimnazjum nr . 6 przy al . pracy sp . nr . 82 przy ul . blacharskiej sp . nr . 109 przy ul . inżynierskiej sp . nr . 15 przy ul . solskiego gimnazjum nr . 40 przy ul . morelowskiego pomoc ma polegać na szukaniu sponsorów którzy sfinansują specjalistyczne szczepienia koszt szczepienia jednego drzewa to ok . 70 zł vat sponsor który uratuje kasztanowca ma zapewnioną na nim tabliczkę reklamową podobna akcja odbyła się w świdnicy gdzie szczepionkę dla 99 drzew zasponsorowała firma imp comfort która kupiła 10 ha . gruntów w mieście i zamierza wybudować tam swoją fabrykę szrotówek kasztanowcowiaczek już od jakiegoś czasu niszczy kasztanowce do niedawna uważano że tego szkodnika można wytępić tylko poprzez palenie i kompostowanie liści kasztanowca teraz wiemy że możemy je uratować poprzez specjalne szczepionki jeśli możesz pomóc wrocławs

#### <span style='font-family:Georgia'> Inserting comas

In [25]:
for conj in con_list_yes:
    data_train['FixedOutput'] = data_train['FixedOutput'].apply(
        lambda x: x.replace(
            ' ' + conj + ' ',
            ', '+conj+' '
        )
    )
data_train['FixedOutput'] = data_train['FixedOutput'].apply(
    lambda x: x.replace('. ,', ',')
)

In [26]:
data_train.loc[0, 'FixedOutput']

'we wrocławiu walkę ze szkodnikiem rozpoczyna zespół szkół budowlanych przy ul . grabiszyńskiej do akcji na ratunek kasztanowcom zsb zachęcił pobliskie szkoły gimnazjum nr . 6 przy al . pracy sp . nr . 82 przy ul . blacharskiej sp . nr . 109 przy ul . inżynierskiej sp . nr . 15 przy ul . solskiego gimnazjum nr . 40 przy ul . morelowskiego pomoc ma polegać na szukaniu sponsorów, którzy sfinansują specjalistyczne szczepienia koszt szczepienia jednego drzewa to ok . 70 zł vat sponsor, który uratuje kasztanowca ma zapewnioną na nim tabliczkę reklamową podobna akcja odbyła się w świdnicy gdzie szczepionkę dla 99 drzew zasponsorowała firma imp comfort, która kupiła 10 ha . gruntów w mieście i zamierza wybudować tam swoją fabrykę szrotówek kasztanowcowiaczek już od jakiegoś czasu niszczy kasztanowce do niedawna uważano, że tego szkodnika można wytępić tylko poprzez palenie i kompostowanie liści kasztanowca teraz wiemy, że możemy je uratować poprzez specjalne szczepionki, jeśli możesz pomóc wr

In [27]:
data_train.FixedOutput = data_train.FixedOutput.apply(lambda x: x.lstrip())

In [28]:
# data_train.to_csv(
#     "../../data/outputs/baseline/excluding_pauses/train_data.csv", index=False
# )

### <span style='font-family:Georgia'> Test data
#### <span style='font-family:Georgia'> Adding fullstops based on timestamps

In [29]:
data_test = read_files("../../data/source/poleval_fa.validation/validation", sep="\t")
data_test_calc = data_test.drop(["timestamp_start", "timestamp_stop"], axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 69.86it/s]


In [30]:
data_test_calc_joined = (
    data_test_calc[["index", "word"]]
    .groupby(["index"])["word"]
    .agg(" ".join)
    .reset_index()
)

In [31]:
for symb in noisy_words_test:
    data_test_calc_joined["word"] = data_test_calc_joined["word"].apply(
        lambda x: x.replace(symb, "")
    )

data_test_calc_joined["word"] = data_test_calc_joined["word"].apply(
    rm_consecutive_spaces
)

for symb in symbols_to_replace_test:
    data_test_calc_joined["word"] = data_test_calc_joined["word"].apply(
        lambda x: x.replace(symb, "")
    )

data_test_calc_joined["word"] = data_test_calc_joined["word"].apply(
    rm_consecutive_spaces
)
data_test_calc_joined.columns = ["FileId", "FixedOutput"]
data_test = copy.deepcopy(data_test_calc_joined)

In [32]:
data_test.shape

(200, 2)

In [33]:
data_test.loc[1, 'FixedOutput']

'prezydent usa george bush powiedział że odnowa nowego orleanu i innych zalanych terenów potrwa lata stan luizjana wraz ze stolicą nowym orleanem ucierpiał w wyniku ataku huraganu katrina prezydent skrócił swoje wakacje w teksasie oraz zwołał posiedzenie gabinetu który obejmie kontrolę nad nadzorem akcji ratunkowej jednej z najgorszych katastrof naturalnych bush wyznaczył trzy priorytety dotyczące walki ze skutkami kataklizmu ratowanie życia ofiarom w tym poszukiwanie zaginionych pomoc poszkodowanym i odnalezionym odbudowa i usuwanie szkód w ramach akcji ratunkowej przydzielono dodatkowo 10 tysięcy żołnierzy którzy wspomogą operację w regionach najbardziej dotkniętych katastrofą zaliczają się do nich części stanów luizjany i mississippi służby medyczne zaoferowały ponad 10 tysięcy łóżek zaś armia amerykańska wysyła swoje helikoptery i łodzie aby uratować mieszkańców znajdujących się w schronieniu późnym wieczorem w środę rozpoczęła się ewakuacja reszty mieszkańców nowego orleanu tych k

#### <span style='font-family:Georgia'> Inserting fullstops in abbreviations

In [34]:
# inserting fullstops after abbreviations + removing doubled spaces
for abbr in abbreviations:
    data_test['FixedOutput'] = data_test['FixedOutput'].apply(
        lambda x: x.replace(
            ' ' + abbr + ' ',
            ' ' + abbr.replace(' ', ' . ') + ' . ')
    )
data_test['FixedOutput'] = data_test['FixedOutput'].apply(
    lambda x: x.replace('. .', '.')
)

In [35]:
data_test.head()

,FileId,FixedOutput
0,wikinews179014,w gdańsku zgodnie już z coroczną czerwcową tra...
1,wikinews179354,prezydent usa george bush powiedział że odnowa...
2,wikinews179650,mamy najgorsze przedmieścia w europie nie woln...
3,wikinews179740,w sejmie trwała dziś debata nad sprawozdaniem ...
4,wikinews179784,aleksander łukaszenka który za 3 miesiące będz...


In [36]:
#example
data_test.loc[1, 'FixedOutput']

'prezydent usa george bush powiedział że odnowa nowego orleanu i innych zalanych terenów potrwa lata stan luizjana wraz ze stolicą nowym orleanem ucierpiał w wyniku ataku huraganu katrina prezydent skrócił swoje wakacje w teksasie oraz zwołał posiedzenie gabinetu który obejmie kontrolę nad nadzorem akcji ratunkowej jednej z najgorszych katastrof naturalnych bush wyznaczył trzy priorytety dotyczące walki ze skutkami kataklizmu ratowanie życia ofiarom w tym poszukiwanie zaginionych pomoc poszkodowanym i odnalezionym odbudowa i usuwanie szkód w ramach akcji ratunkowej przydzielono dodatkowo 10 tysięcy żołnierzy którzy wspomogą operację w regionach najbardziej dotkniętych katastrofą zaliczają się do nich części stanów luizjany i mississippi służby medyczne zaoferowały ponad 10 tysięcy łóżek zaś armia amerykańska wysyła swoje helikoptery i łodzie aby uratować mieszkańców znajdujących się w schronieniu późnym wieczorem w środę rozpoczęła się ewakuacja reszty mieszkańców nowego orleanu tych k

#### <span style='font-family:Georgia'> Inserting comas

In [37]:
for conj in con_list_yes:
    data_test['FixedOutput'] = data_test['FixedOutput'].apply(
        lambda x: x.replace(
            ' ' + conj + ' ',
            ', ' + conj + ' ')
    )
data_test['FixedOutput'] = data_test['FixedOutput'].apply(
    lambda x: x.replace('. ,', ',')
)

In [38]:
#example result
data_test.loc[1, 'FixedOutput']

'prezydent usa george bush powiedział, że odnowa nowego orleanu i innych zalanych terenów potrwa lata stan luizjana wraz ze stolicą nowym orleanem ucierpiał w wyniku ataku huraganu katrina prezydent skrócił swoje wakacje w teksasie oraz zwołał posiedzenie gabinetu, który obejmie kontrolę nad nadzorem akcji ratunkowej jednej z najgorszych katastrof naturalnych bush wyznaczył trzy priorytety dotyczące walki ze skutkami kataklizmu ratowanie życia ofiarom w tym poszukiwanie zaginionych pomoc poszkodowanym i odnalezionym odbudowa i usuwanie szkód w ramach akcji ratunkowej przydzielono dodatkowo 10 tysięcy żołnierzy, którzy wspomogą operację w regionach najbardziej dotkniętych katastrofą zaliczają się do nich części stanów luizjany i mississippi służby medyczne zaoferowały ponad 10 tysięcy łóżek zaś armia amerykańska wysyła swoje helikoptery i łodzie, aby uratować mieszkańców znajdujących się w schronieniu późnym wieczorem w środę rozpoczęła się ewakuacja reszty mieszkańców nowego orleanu ty

In [39]:
data_test.FixedOutput = data_test.FixedOutput.apply(lambda x: x.lstrip())

In [40]:
# data_test.to_csv('../../data/outputs/baseline/excluding_pauses/test_data.csv', index=False)